# LJT Reports

#### Analyst: Dhruv Singh <br> Report Name: LJT Report <br> Report Quarter, Year: FY 2020 Q1-Q4 <br> Date Updated: 08/10/2021

##### Date filter

In [ ]:
# enter start and end date in "mm/dd/yyyy" format
start_date = "10/01/2019"
end_date = "09/30/2020"

In [ ]:
# enter FY and quarter(s) for report: used in graph labels
year = '2020'
quarter = 'Q1-Q4'

In [ ]:
# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Data Cleaning

In [ ]:
# reading in data
ljt = pd.read_csv('data/services.csv', dtype = 'unicode')

##### Checking for Test Cases

In [ ]:
# subsetting to exclude test cases
ljt = ljt[(ljt['FirstName'] != "Kurt") & (ljt['LastName'] != "Hummel")]

##### Replacing missing values

In [ ]:
ljt = ljt.fillna(0)

##### Recoding Program: WIOA & Local Training, to FEMS and MPD

In [ ]:
# recoding program title from WIOA to FEMS, for FEMS
ljt.loc[ljt["ServiceProviderName"] == "DC Department of Fire & Emergency Services", "ProgramTitle"] = "DC Department of Fire & Emergency Services"

In [ ]:
# recoding program title from WIOA to MPD, for MPD
ljt.loc[ljt["ServiceProviderName"] == "MPD (Cadet Corps Branch)", "ProgramTitle"] = "MPD (Cadet Corps Branch)"

##### Subsetting columns

In [ ]:
# subsetting columns
ljt = ljt[["StateId", "OfficeName", "ProgramTitle", "FirstName", "LastName", "ServiceCode", "ActualstartDate", "ActualEndDate", "CompletionStatus", "ServiceProviderName", "CourseName", "CredentialType1", "CredentialDate1", "CredentialType2", "CredentialDate2", "CredentialType3", "CredentialDate3", "CredentialType4", "CredentialDate4", "EmploymentStartDate"]]

##### Binarizing date and completions

In [ ]:
# converting actual and end date to datetime 
ljt.ActualstartDate = pd.to_datetime(ljt.ActualstartDate)
ljt.ActualEndDate = pd.to_datetime(ljt.ActualEndDate)

In [ ]:
# extracting month and year columns
ljt['StartMonth'] = pd.DatetimeIndex(ljt['ActualstartDate']).month
ljt['StartYear'] = pd.DatetimeIndex(ljt['ActualstartDate']).year
ljt['EndMonth'] = pd.DatetimeIndex(ljt['ActualEndDate']).month
ljt['EndYear'] = pd.DatetimeIndex(ljt['ActualEndDate']).year

# converting columns float to int
ljt['EndMonth'] = ljt['EndMonth'].apply(np.int64)
ljt['EndYear'] = ljt['EndYear'].apply(np.int64)

# creating month-year columns
ljt['StartMonthYear'] = ljt.StartYear.map(str) + "-" + ljt.StartMonth.map(str)
ljt['EndMonthYear'] = ljt.EndYear.map(str) + "-" + ljt.EndMonth.map(str)

In [ ]:
# binarizing start date and end date columns
ljt['StartDate'] = 0
ljt['EndDate'] = 0

# recoding binary date columns based on date filter
ljt.loc[(ljt.ActualstartDate > start_date) & (ljt.ActualstartDate < end_date), 'StartDate'] = 1
ljt.loc[(ljt.ActualEndDate > start_date) & (ljt.ActualEndDate < end_date), 'EndDate'] = 1

In [ ]:
# binarizing completion status column
ljt['Completion_num'] = 0

# recoding completion status columns based on date filter
ljt.loc[ljt.CompletionStatus == 'Successful Completion', 'Completion_num'] = 1

##### Subsetting rows

In [ ]:
# subsetting rows: programs
ljt = ljt[(ljt['ProgramTitle'] == "Pre-Apprenticeship Program") | (ljt['ProgramTitle'] == "Transitional Employment Program (Way to Work)") | (ljt['ProgramTitle'] == "DC Career Connections (DCCC)") | (ljt['ProgramTitle'] == "DC Infrastructure Academy") | (ljt['ProgramTitle'] == "L.E.A.P. (Learn, Earn, Advance, Prosper)") | (ljt['ProgramTitle'] == "Back to Work 50+ DC") | (ljt['ProgramTitle'] == "MPD (Cadet Corps Branch)") | (ljt['ProgramTitle'] == "DC Department of Fire & Emergency Services")]

In [ ]:
# subsetting rows: date
ljt = ljt[(ljt['StartDate'] == 1) | (ljt['EndDate'] == 1)]

#### De-Duplication

In [ ]:
ljt = ljt.sort_values(by=['StateId', 'FirstName', 'LastName', 'StartDate', 'CompletionStatus'], ascending = (True, True, True, False, True))

##### Splitting dataframe by activity code types

In [ ]:
ljt.shape

In [ ]:
# 200 codes
programs1 = ljt.ProgramTitle.unique().tolist()

In [ ]:
activity_code1 = ["220", "214", "219", "893"]

In [ ]:
# creating code type category column
ljt['code_types'] = 0

# recoding code category columns based on date filter
ljt.loc[(ljt.ProgramTitle.isin(programs1)) & (ljt.ServiceCode.isin(activity_code1)), 'code_types'] = 1

In [ ]:
ljt[ljt.code_types==1].shape

In [ ]:
# 300 codes
programs2 = ['DC Career Connections (DCCC)', 'DC Infrastructure Academy', 'Transitional Employment Program (Way to Work)', 'Pre-Apprenticeship Program', 'MPD (Cadet Corps Branch)', 'DC Department of Fire & Emergency Services']

In [ ]:
activity_code2 = ["328", "313", "300", "315", "333"]

In [ ]:
# recoding code category columns based on date filter
ljt.loc[(ljt.ProgramTitle.isin(programs2)) & (ljt.ServiceCode.isin(activity_code2)), 'code_types'] = 2

In [ ]:
ljt[ljt.code_types==2].shape

#### Step 1: 200 codes

In [ ]:
# outputting dups 1
dups1 = ljt[ljt.code_types==1]
dups1 = dups1[dups1.duplicated(subset = ["ProgramTitle", "ServiceCode", "StateId"], keep=False)]

In [ ]:
dups1.to_csv('data/duplicates1.csv')

In [ ]:
# dropping dups 1: keeping most recent record since most often progression is towards success
ljt1 = ljt[ljt.code_types==1].drop_duplicates(subset = ["ProgramTitle", "ServiceCode", "StateId"], keep='first')

In [ ]:
ljt1.shape

##### Step 2: 300 codes

In [ ]:
# outputting dups 2
dups2 = ljt[ljt.code_types==2]
dups2 = dups2[dups2.duplicated(subset = ["ProgramTitle", "ServiceCode", "StateId", "ServiceProviderName"], keep=False)]

In [ ]:
dups2.to_csv('data/duplicates2.csv')

In [ ]:
# dropping dups 2: keeping most recent record since most often progression is towards success
ljt2 = ljt[ljt.code_types==2].drop_duplicates(subset = ["ProgramTitle", "ServiceCode", "StateId", "ServiceProviderName"], keep='first')

In [ ]:
ljt2.shape

##### Step 3: Remaining Codes

In [ ]:
ljt3 = ljt[ljt.code_types==0]

In [ ]:
ljt = pd.concat([ljt3, ljt1, ljt2])

In [ ]:
ljt.shape

#### Adding abbreviations for Programs and Service Vendors

In [ ]:
# creating abbreviations column
ljt['Abbreviations'] = ''

# recoding abbreviations based on program title
ljt.loc[ljt.ProgramTitle == 'Transitional Employment Program (Way to Work)', 'Abbreviations'] = 'TEP'
ljt.loc[ljt.ProgramTitle == 'Pre-Apprenticeship Program', 'Abbreviations'] = 'Pre-App'
ljt.loc[ljt.ProgramTitle == 'L.E.A.P. (Learn, Earn, Advance, Prosper)', 'Abbreviations'] = 'LEAP'
ljt.loc[ljt.ProgramTitle == 'DC Infrastructure Academy', 'Abbreviations'] = 'DCIA'
ljt.loc[ljt.ProgramTitle == 'DC Career Connections (DCCC)', 'Abbreviations'] = 'DCCC'
ljt.loc[ljt.ProgramTitle == 'Back to Work 50+ DC', 'Abbreviations'] = 'BTW'
ljt.loc[ljt.ProgramTitle == 'MPD (Cadet Corps Branch)', 'Abbreviations'] = 'MPD'
ljt.loc[ljt.ProgramTitle == 'DC Department of Fire & Emergency Services', 'Abbreviations'] = 'FEMS'

In [ ]:
# creating abbreviations 2 column
ljt['Abbreviations2'] = ''

ljt.loc[ljt.ServiceProviderName == 'DC Department of Employment Services', 'Abbreviations2'] = 'DOES'
# ljt.loc[ljt.ServiceProviderName == '0', 'Abbreviations2'] = '0'
ljt.loc[ljt.ServiceProviderName == 'Southeast Veterans Service Center', 'Abbreviations2'] = 'SEVC'
ljt.loc[ljt.ServiceProviderName == 'Wheeler Terrace Apartments', 'Abbreviations2'] = 'WTA'
ljt.loc[ljt.ServiceProviderName == 'CIH Properties', 'Abbreviations2'] = 'CIH'
ljt.loc[ljt.ServiceProviderName == "Pinkie's E.A.T.S", 'Abbreviations2'] = 'PE'
ljt.loc[ljt.ServiceProviderName == 'Woodland Creek Manor', 'Abbreviations2'] = 'WCM'
ljt.loc[ljt.ServiceProviderName == 'Southeast Ministry', 'Abbreviations2'] = 'SEM'
ljt.loc[ljt.ServiceProviderName == 'Skyland Workforce Center', 'Abbreviations2'] = 'SWC'
ljt.loc[ljt.ServiceProviderName == 'Edgewood Management Corporation | Vantage Management', 'Abbreviations2'] = 'EMCVM'
ljt.loc[ljt.ServiceProviderName == 'US Department of Veterans Affairs', 'Abbreviations2'] = 'DVA'
ljt.loc[ljt.ServiceProviderName == "Kirstin's Haven Inc", 'Abbreviations2'] = 'KHI'
ljt.loc[ljt.ServiceProviderName == 'Stanley Foods', 'Abbreviations2'] = 'SF'
ljt.loc[ljt.ServiceProviderName == 'Asbury Dwellings Apartments - Edgewood', 'Abbreviations2'] = 'ADAE'
ljt.loc[ljt.ServiceProviderName == 'Trinity Plaza', 'Abbreviations2'] = 'TP'
ljt.loc[ljt.ServiceProviderName == 'Puddin LLC', 'Abbreviations2'] = 'PDN'
ljt.loc[ljt.ServiceProviderName == 'Robinsons, LLC', 'Abbreviations2'] = 'RBSN'
ljt.loc[ljt.ServiceProviderName == '2321 4th Street NE, LLC', 'Abbreviations2'] = '4ST'
ljt.loc[ljt.ServiceProviderName == 'Bread for the City - Southeast Center', 'Abbreviations2'] = 'BCSC'
ljt.loc[ljt.ServiceProviderName == 'Capitol Park Plaza Apartments', 'Abbreviations2'] = 'CPPA'
ljt.loc[ljt.ServiceProviderName == 'Worthington Woods Apartments (Residential One)', 'Abbreviations2'] = 'WWA'
ljt.loc[ljt.ServiceProviderName == 'VOW Transportation, LLC', 'Abbreviations2'] = 'VOW'
ljt.loc[ljt.ServiceProviderName == 'Career Path DC', 'Abbreviations2'] = 'CPDC'
ljt.loc[ljt.ServiceProviderName == 'Friendship Court Apartments', 'Abbreviations2'] = 'FCA'
ljt.loc[ljt.ServiceProviderName == 'Carver Terrace, LP', 'Abbreviations2'] = 'CTLP'
ljt.loc[ljt.ServiceProviderName == 'W Street Apartments', 'Abbreviations2'] = 'WSA'
ljt.loc[ljt.ServiceProviderName == 'Tastemakers', 'Abbreviations2'] = 'TSMK'
ljt.loc[ljt.ServiceProviderName == 'Residential One, LLC', 'Abbreviations2'] = 'RO'
ljt.loc[ljt.ServiceProviderName == 'Huntwood Courts', 'Abbreviations2'] = 'HC'
ljt.loc[ljt.ServiceProviderName == 'DC Department of Housing & Community Development', 'Abbreviations2'] = 'HCD'
ljt.loc[ljt.ServiceProviderName == 'Nannie Helen Apartments', 'Abbreviations2'] = 'NHA'
ljt.loc[ljt.ServiceProviderName == 'Metropolitan Community Church of Washington DC', 'Abbreviations2'] = 'MCCW'
ljt.loc[ljt.ServiceProviderName == 'Child and Family Services', 'Abbreviations2'] = 'CFS'
ljt.loc[ljt.ServiceProviderName == 'Medstar-Washington Hospital Center', 'Abbreviations2'] = 'MWHC'
ljt.loc[ljt.ServiceProviderName == 'Cascade Apartments', 'Abbreviations2'] = 'CA'
ljt.loc[ljt.ServiceProviderName == 'Trusted Solutions Group', 'Abbreviations2'] = 'TSG'
ljt.loc[ljt.ServiceProviderName == 'Brookland Manor Community Center', 'Abbreviations2'] = 'BMCC'
ljt.loc[ljt.ServiceProviderName == 'Lynn Property - 28th Street', 'Abbreviations2'] = 'LPS'
ljt.loc[ljt.ServiceProviderName == 'MAYFAIR MANSIONS APARTMENTS', 'Abbreviations2'] = 'MMA'
ljt.loc[ljt.ServiceProviderName == 'KRA Corporation-DC', 'Abbreviations2'] = 'KRA'
ljt.loc[ljt.ServiceProviderName == 'Mayors Office Of Latino Affairs', 'Abbreviations2'] = 'MOLA'
ljt.loc[ljt.ServiceProviderName == 'Faria Management - Rhode Island Ave.', 'Abbreviations2'] = 'FMRA'
ljt.loc[ljt.ServiceProviderName == 'Capital Service Management', 'Abbreviations2'] = 'CSM'
ljt.loc[ljt.ServiceProviderName == 'University of District of Columbia', 'Abbreviations2'] = 'UDC'
ljt.loc[ljt.ServiceProviderName == "Agape Woodland Tiger Children's Academy", 'Abbreviations2'] = 'AWTA'
ljt.loc[ljt.ServiceProviderName == 'Severna on K Street', 'Abbreviations2'] = 'SKS'
ljt.loc[ljt.ServiceProviderName == 'Congress Heights Community Training & Development (WEX)', 'Abbreviations2'] = 'CHTD'
ljt.loc[ljt.ServiceProviderName == 'Anacostia High School', 'Abbreviations2'] = 'AHS'
ljt.loc[ljt.ServiceProviderName == 'Capital Area Food Bank (WEX)', 'Abbreviations2'] = 'CAFB'
ljt.loc[ljt.ServiceProviderName == "Momma' Safe Haven", 'Abbreviations2'] = 'MSH'
ljt.loc[ljt.ServiceProviderName == 'King Towers Apartments', 'Abbreviations2'] = 'KTA'
ljt.loc[ljt.ServiceProviderName == 'Madison Courts Apartments', 'Abbreviations2'] = 'MCA'
ljt.loc[ljt.ServiceProviderName == 'Berliner Specialty Distributors', 'Abbreviations2'] = 'BSD'
ljt.loc[ljt.ServiceProviderName == 'Loving Care Day Nursery', 'Abbreviations2'] = 'LCDN'
ljt.loc[ljt.ServiceProviderName == 'Community Forklift', 'Abbreviations2'] = 'CF'
ljt.loc[ljt.ServiceProviderName == 'Alpha Omega Properties (WEX)', 'Abbreviations2'] = 'AOP'
ljt.loc[ljt.ServiceProviderName == 'Belmont Crossing Apartments', 'Abbreviations2'] = 'BCA'
ljt.loc[ljt.ServiceProviderName == 'Total Office Products', 'Abbreviations2'] = 'TOP'
ljt.loc[ljt.ServiceProviderName == 'Park 7 Apartments', 'Abbreviations2'] = 'PA'
ljt.loc[ljt.ServiceProviderName == "AL's Twin Air LLC", 'Abbreviations2'] = 'ATAL'
ljt.loc[ljt.ServiceProviderName == 'Foster House Apartments', 'Abbreviations2'] = 'FHA'
ljt.loc[ljt.ServiceProviderName == 'Southern Homes and Gardens Cooperative', 'Abbreviations2'] = 'SHGC'
ljt.loc[ljt.ServiceProviderName == 'DowntownDC Business Improvement District (BID)', 'Abbreviations2'] = 'BID'
ljt.loc[ljt.ServiceProviderName == 'Hyatts Place DC National Mall', 'Abbreviations2'] = 'HPNM'
ljt.loc[ljt.ServiceProviderName == 'Lawyers Committee for Civil Rights under Law', 'Abbreviations2'] = 'LCCR'
ljt.loc[ljt.ServiceProviderName == 'Thee TraRon Center', 'Abbreviations2'] = 'TTC'
ljt.loc[ljt.ServiceProviderName == 'First Baptist Senior Center', 'Abbreviations2'] = 'FBSC'
ljt.loc[ljt.ServiceProviderName == 'Arbor View Apartments', 'Abbreviations2'] = 'AVA'
ljt.loc[ljt.ServiceProviderName == 'Department of Public Works', 'Abbreviations2'] = 'DPW'
ljt.loc[ljt.ServiceProviderName == 'Penn Mar Apartments-Residential 1', 'Abbreviations2'] = 'PMA'
ljt.loc[ljt.ServiceProviderName == 'Benning Park Apartments (WEX)', 'Abbreviations2'] = 'BPA'
ljt.loc[ljt.ServiceProviderName == 'The Village Cafe', 'Abbreviations2'] = 'TVC'
ljt.loc[ljt.ServiceProviderName == 'Brightwood - Concord Apartments', 'Abbreviations2'] = 'BRCA'
ljt.loc[ljt.ServiceProviderName == 'Therapeutic Sessions', 'Abbreviations2'] = 'TS'
ljt.loc[ljt.ServiceProviderName == 'Omni Excavators Inc', 'Abbreviations2'] = 'OEI'
ljt.loc[ljt.ServiceProviderName == 'Highland Ridge Leasing', 'Abbreviations2'] = 'HRL'
ljt.loc[ljt.ServiceProviderName == "Chad O'L Public Relations & Events, LLC", 'Abbreviations2'] = 'COPR'
ljt.loc[ljt.ServiceProviderName == 'New Bethel Baptist Church', 'Abbreviations2'] = 'NBBC'
ljt.loc[ljt.ServiceProviderName == 'Sharcon Management', 'Abbreviations2'] = 'SM'
ljt.loc[ljt.ServiceProviderName == 'Washington Hospital Center', 'Abbreviations2'] = 'WHC'
ljt.loc[ljt.ServiceProviderName == 'Department of Energy and Environment', 'Abbreviations2'] = 'DEE'
ljt.loc[ljt.ServiceProviderName == 'House of Lebanon (Columbus Properties)', 'Abbreviations2'] = 'HLCP'
ljt.loc[ljt.ServiceProviderName == 'Department of Employment Services', 'Abbreviations2'] = 'DOES'
ljt.loc[ljt.ServiceProviderName == 'Boys and Girls Club - The ARC', 'Abbreviations2'] = 'BGC'
ljt.loc[ljt.ServiceProviderName == 'Fresh Food Factory Market', 'Abbreviations2'] = 'FFFM'
ljt.loc[ljt.ServiceProviderName == 'DC Office of Veteran Affairs', 'Abbreviations2'] = 'OVA'
ljt.loc[ljt.ServiceProviderName == 'Dc City Smokehouse', 'Abbreviations2'] = 'DCCS'
ljt.loc[ljt.ServiceProviderName == 'WDC Solar, Inc.', 'Abbreviations2'] = 'WDC'
ljt.loc[ljt.ServiceProviderName == 'Interns4Hire', 'Abbreviations2'] = 'I4H'
ljt.loc[ljt.ServiceProviderName == 'DC Department of General Services', 'Abbreviations2'] = 'DGS'
ljt.loc[ljt.ServiceProviderName == 'Glendale Plaza', 'Abbreviations2'] = 'GP'
ljt.loc[ljt.ServiceProviderName == 'Washington Parks and People', 'Abbreviations2'] = 'PAP'
ljt.loc[ljt.ServiceProviderName == 'United Planning Organization (UPO)', 'Abbreviations2'] = 'UPO'
ljt.loc[ljt.ServiceProviderName == 'Health Resources Services - Intake Center', 'Abbreviations2'] = 'HRS'
ljt.loc[ljt.ServiceProviderName == 'Shining Star Early Learning Center', 'Abbreviations2'] = 'SSEL'
ljt.loc[ljt.ServiceProviderName == 'US Department Veterans Affairs', 'Abbreviations2'] = 'DVA'
ljt.loc[ljt.ServiceProviderName == 'One Care DC', 'Abbreviations2'] = 'OCDC'
ljt.loc[ljt.ServiceProviderName == 'DC Re-Engagement Center', 'Abbreviations2'] = 'REC'
ljt.loc[ljt.ServiceProviderName == 'DC Central Kitchen 2 (WEX)', 'Abbreviations2'] = 'DCC2'
ljt.loc[ljt.ServiceProviderName == 'Anchor Mental Health of Catholic Charities', 'Abbreviations2'] = 'AMH'
ljt.loc[ljt.ServiceProviderName == 'Homes for hopes', 'Abbreviations2'] = 'HFH'
ljt.loc[ljt.ServiceProviderName == 'Excel Automotive Institute', 'Abbreviations2'] = 'EAI'
ljt.loc[ljt.ServiceProviderName == 'Veterans Affairs Medical Center (VA Hospital)', 'Abbreviations2'] = 'VAH'
ljt.loc[ljt.ServiceProviderName == 'Office of Returning Citizens', 'Abbreviations2'] = 'ORC'
ljt.loc[ljt.ServiceProviderName == 'National Service Contractors Inc', 'Abbreviations2'] = 'NSI'
ljt.loc[ljt.ServiceProviderName == 'DC City Council', 'Abbreviations2'] = 'DCCC'
ljt.loc[ljt.ServiceProviderName == 'Dudley Beauty College', 'Abbreviations2'] = 'DBC'
ljt.loc[ljt.ServiceProviderName == 'Atlantic Terrace Apartments', 'Abbreviations2'] = 'ATA'
ljt.loc[ljt.ServiceProviderName == 'Plum Good', 'Abbreviations2'] = 'PG'
ljt.loc[ljt.ServiceProviderName == 'DC L.E.A.P. Academy', 'Abbreviations2'] = 'LEAP'
ljt.loc[ljt.ServiceProviderName == 'Art and Soul Solutions, Inc.', 'Abbreviations2'] = 'ASSI'
ljt.loc[ljt.ServiceProviderName == 'Mount Carmel Baptist Church', 'Abbreviations2'] = 'MCBC'
ljt.loc[ljt.ServiceProviderName == 'The Village Evergreen', 'Abbreviations2'] = 'TVE'
ljt.loc[ljt.ServiceProviderName == 'LPJ Legal PLLC', 'Abbreviations2'] = 'LPJ'
ljt.loc[ljt.ServiceProviderName == 'DC Doors, Inc.', 'Abbreviations2'] = 'DCDI'
ljt.loc[ljt.ServiceProviderName == 'Sustainable Facilities Management Services', 'Abbreviations2'] = 'SFMS'
ljt.loc[ljt.ServiceProviderName == 'Nursing Assistant Academy', 'Abbreviations2'] = 'NAA'
ljt.loc[ljt.ServiceProviderName == 'Voices for a Second Chance', 'Abbreviations2'] = 'VSC'
ljt.loc[ljt.ServiceProviderName == 'Early Stages', 'Abbreviations2'] = 'ES'
ljt.loc[ljt.ServiceProviderName == 'Hyatt Place', 'Abbreviations2'] = 'HP'
ljt.loc[ljt.ServiceProviderName == 'Maya Angelou School-See Forever Foundation', 'Abbreviations2'] = 'MASF'
ljt.loc[ljt.ServiceProviderName == 'May Lightfoot', 'Abbreviations2'] = 'ML'
ljt.loc[ljt.ServiceProviderName == 'Office of the Chief Technology Office', 'Abbreviations2'] = 'OCTO'
ljt.loc[ljt.ServiceProviderName == 'Residence Inn Arlington Capital View', 'Abbreviations2'] = 'RIA'
ljt.loc[ljt.ServiceProviderName == 'Promoting Love and Wisdom', 'Abbreviations2'] = 'POW'
ljt.loc[ljt.ServiceProviderName == 'The National Children Center', 'Abbreviations2'] = 'NCC'
ljt.loc[ljt.ServiceProviderName == 'Equity Management II LLC', 'Abbreviations2'] = 'EM'
ljt.loc[ljt.ServiceProviderName == 'CC Prep - MLK', 'Abbreviations2'] = 'CCML'
ljt.loc[ljt.ServiceProviderName == '1st CDL Training Center of NOVA', 'Abbreviations2'] = 'CDL'
ljt.loc[ljt.ServiceProviderName == 'Georgetown University', 'Abbreviations2'] = 'GU'
ljt.loc[ljt.ServiceProviderName == 'So Others Might Eat (SOME)', 'Abbreviations2'] = 'SOME'
ljt.loc[ljt.ServiceProviderName == 'Metropolitan Washington Council AFL-CIO', 'Abbreviations2'] = 'AFL'
ljt.loc[ljt.ServiceProviderName == 'Washington Literacy Center', 'Abbreviations2'] = 'WLC'
ljt.loc[ljt.ServiceProviderName == 'Washington Area New Automobile Dealers Association (WANADA)', 'Abbreviations2'] = 'WANA'
ljt.loc[ljt.ServiceProviderName == 'MPD (Cadet Corps Branch)', 'Abbreviations2'] = 'MPD'
ljt.loc[ljt.ServiceProviderName == 'University of the District of Columbia (UDC)', 'Abbreviations2'] = 'UDC'
ljt.loc[ljt.ServiceProviderName == 'CNI, LLC', 'Abbreviations2'] = 'CNI'
ljt.loc[ljt.ServiceProviderName == 'Opportunities Industrialization Center of Washington-OIC/DC', 'Abbreviations2'] = 'OIC'
ljt.loc[ljt.ServiceProviderName == 'Pepco Holdings, Inc.', 'Abbreviations2'] = 'PEP'
ljt.loc[ljt.ServiceProviderName == 'GRID Alternatives Mid-Atlantic', 'Abbreviations2'] = 'GRID'
ljt.loc[ljt.ServiceProviderName == 'Hamel Builders Inc.', 'Abbreviations2'] = 'HBI'
ljt.loc[ljt.ServiceProviderName == 'The Training Zone of the DMV', 'Abbreviations2'] = 'TTZ'
ljt.loc[ljt.ServiceProviderName == 'DC Department of Fire & Emergency Services', 'Abbreviations2'] = 'FEMS'
ljt.loc[ljt.ServiceProviderName == 'Latin American Youth Center-LAYC', 'Abbreviations2'] = 'LAYC'
ljt.loc[ljt.ServiceProviderName == 'KBEC Group, Inc.', 'Abbreviations2'] = 'KBEC'
ljt.loc[ljt.ServiceProviderName == 'Toni Thomas Associates', 'Abbreviations2'] = 'TTA'
ljt.loc[ljt.ServiceProviderName == 'Westlink Career Institute', 'Abbreviations2'] = 'WCI'
ljt.loc[ljt.ServiceProviderName == 'Served Academy', 'Abbreviations2'] = 'SA'
ljt.loc[ljt.ServiceProviderName == 'Run Hope Work' 'Abbreviations2'] = 'RHW'

## Report 1: Job Readiness Training

#### Filter 1: Program (TEP, DCCC, DCIA, BTW 50+)

In [ ]:
# filtering to relevant programs
jrt = ljt[(ljt.ProgramTitle == 'Transitional Employment Program (Way to Work)') | (ljt.ProgramTitle == 'DC Career Connections (DCCC)') | (ljt.ProgramTitle == 'DC Infrastructure Academy') | (ljt.ProgramTitle == 'Back to Work 50+ DC')]

#### Filter 2: Activity Code (220)

In [ ]:
# filtering by activity code
jrt = jrt[jrt.ServiceCode == '220']

### Report 1.1: Enrollments

#### Filter 3: Start Date

In [ ]:
# filtering by start date
jrt_st = jrt[jrt.StartDate == 1]

In [ ]:
# creating enrollments table
jrt_1 = jrt_st[['Abbreviations', 'Completion_num']].groupby('Abbreviations').count()
jrt_1 = jrt_1.rename(columns = {"Completion_num": "Enrollments"})
jrt_1

#### Visualizations 1.1: Enrollments

In [ ]:
plt.style.use('ggplot')

x = jrt_1.index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, jrt_1['Enrollments'], color='green')
plt.xlabel("Program")
plt.ylabel("Enrollments")
plt.title("Enrollments for Job Readiness Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)

plt.savefig('graphs/graph1.1_jrt_enr.png', dpi=300, bbox_inches='tight')
plt.show()

### Report 1.2: Completions

#### Filter 4: End Date

In [ ]:
# filtering by end date
jrt_en = jrt[jrt.EndDate == 1]

In [ ]:
# creating completions table
jrt_2 = jrt_en[['Abbreviations', 'Completion_num']].groupby("Abbreviations").agg(['count', 'sum'])
jrt_2.columns = jrt_2.columns.get_level_values(1)
jrt_2 = jrt_2.rename(columns = {"count": "Total", "sum": "Completions"})
jrt_2['Completion_Ratio'] = (jrt_2.Completions / jrt_2.Total)*100
jrt_2

#### Visualizations 1.2: Completions

In [ ]:
X = jrt_2.index.tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, jrt_2['Total'], 0.4, label = 'Total')
plt.bar(X_axis + 0.2, jrt_2['Completions'], 0.4, label = 'Successful Completions')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Successful Completions of Job Readiness Training FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph1.2_jrt_comp.png', dpi=300, bbox_inches='tight')
plt.show()

#### Visualization 1.3: Completion Rates

In [ ]:
plt.style.use('ggplot')

x = jrt_2.index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, jrt_2['Completion_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Successful Completions (%)")
plt.title("Successful Completion Rates for Job Readiness Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph1.3_jrt_comp_pct.png', dpi=300, bbox_inches='tight')
plt.show()

## Report 2: Occupational Skills Training

#### Filter 1: Program (TEP, DCCC, DCIA, FEMS, MPD, Pre-App)

In [ ]:
# filtering to relevant programs
ost = ljt[(ljt.ProgramTitle == 'Transitional Employment Program (Way to Work)') | (ljt.ProgramTitle == 'DC Career Connections (DCCC)') | (ljt.ProgramTitle == 'DC Infrastructure Academy') | (ljt.ServiceProviderName == 'DC Department of Fire & Emergency Services') | (ljt.ServiceProviderName == 'MPD (Cadet Corps Branch)') | (ljt.ProgramTitle == 'Pre-Apprenticeship Program')]

#### Filter 2: Activity Code (315, 328, 313)

In [ ]:
# filtering by activity code
ost = ost[(ost.ServiceCode == '315') | (ost.ServiceCode == '328') | (ost.ServiceCode == '333')]

### Report 2.1: Enrollments

#### Filter 3: Start Date

In [ ]:
# filtering by start date
ost_st = ost[ost.StartDate == 1]

In [ ]:
ljt.head()

In [ ]:
# creating enrollments table
ost_1 = ost_st[['ProgramTitle', 'Abbreviations2', 'Completion_num']].groupby(['ProgramTitle', 'Abbreviations2']).count()
ost_1 = ost_1.rename(columns = {"Completion_num": "Enrollments"})
ost_1 = ost_1.reset_index(level='ProgramTitle')
ost_1

#### Visualizations 2.1: Enrollments

In [ ]:
plt.style.use('ggplot')

x = ost_1[ost_1.ProgramTitle == 'DC Career Connections (DCCC)'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_1[ost_1.ProgramTitle == 'DC Career Connections (DCCC)']['Enrollments'], color='green')
plt.xlabel("Vendor")
plt.ylabel("Enrollments")
plt.title("Enrollments for DCCC Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.1_ost_dccc_enr.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = ost_1[ost_1.ProgramTitle == 'DC Department of Fire & Emergency Services'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_1[ost_1.ProgramTitle == 'DC Department of Fire & Emergency Services']['Enrollments'], color='green')
plt.xlabel("Vendor")
plt.ylabel("Enrollments")
plt.title("Enrollments for FEMS Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.1_ost_fems_enr.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = ost_1[ost_1.ProgramTitle == 'DC Infrastructure Academy'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_1[ost_1.ProgramTitle == 'DC Infrastructure Academy']['Enrollments'], color='green')
plt.xlabel("Vendor")
plt.ylabel("Enrollments")
plt.title("Enrollments for DCIA Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.1_ost_dcia_enr.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = ost_1[ost_1.ProgramTitle == 'MPD (Cadet Corps Branch)'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_1[ost_1.ProgramTitle == 'MPD (Cadet Corps Branch)']['Enrollments'], color='green')
plt.xlabel("Vendor")
plt.ylabel("Enrollments")
plt.title("Enrollments for MPD Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.1_ost_mpd_enr.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = ost_1[ost_1.ProgramTitle == 'Pre-Apprenticeship Program'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_1[ost_1.ProgramTitle == 'Pre-Apprenticeship Program']['Enrollments'], color='green')
plt.xlabel("Vendor")
plt.ylabel("Enrollments")
plt.title("Enrollments for Pre-App Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.1_ost_pre_app_enr.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = ost_1[ost_1.ProgramTitle == 'Transitional Employment Program (Way to Work)'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_1[ost_1.ProgramTitle == 'Transitional Employment Program (Way to Work)']['Enrollments'], color='green')
plt.xlabel("Vendor")
plt.ylabel("Enrollments")
plt.title("Enrollments for TEP Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.1_ost_tep_enr.png', dpi=300, bbox_inches='tight')
plt.show()

### Report 2.2: Completions

In [ ]:
# filtering by end date
ost_en = ost[ost.EndDate == 1]

In [ ]:
# creating completions table
ost_2 = ost_en[['ProgramTitle', 'Abbreviations2', 'Completion_num']].groupby(["ProgramTitle", "Abbreviations2"]).agg(['count', 'sum'])
ost_2.columns = ost_2.columns.get_level_values(1)
ost_2 = ost_2.rename(columns = {"count": "Total", "sum": "Completions"})
ost_2['Completion_Ratio'] = (ost_2.Completions / ost_2.Total)*100
ost_2 = ost_2.reset_index(level='ProgramTitle')
ost_2

#### Visualizations 2.2: Completions

In [ ]:
X = ost_2[ost_2.ProgramTitle == 'DC Career Connections (DCCC)'].index.tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, ost_2[ost_2.ProgramTitle == 'DC Career Connections (DCCC)']['Total'], 0.4, label = 'Total')
plt.bar(X_axis + 0.2, ost_2[ost_2.ProgramTitle == 'DC Career Connections (DCCC)']['Completions'], 0.4, label = 'Successful Completions')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Successful Completions of DCCC Occupational Skills Training FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph2.2_ost_dccc_comp.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
X = ost_2[ost_2.ProgramTitle == 'DC Department of Fire & Emergency Services'].index.tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, ost_2[ost_2.ProgramTitle == 'DC Department of Fire & Emergency Services']['Total'], 0.4, label = 'Total')
plt.bar(X_axis + 0.2, ost_2[ost_2.ProgramTitle == 'DC Department of Fire & Emergency Services']['Completions'], 0.4, label = 'Successful Completions')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Successful Completions of FEMS Occupational Skills Training FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph2.2_ost_fems_comp.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
X = ost_2[ost_2.ProgramTitle == 'DC Infrastructure Academy'].index.tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, ost_2[ost_2.ProgramTitle == 'DC Infrastructure Academy']['Total'], 0.4, label = 'Total')
plt.bar(X_axis + 0.2, ost_2[ost_2.ProgramTitle == 'DC Infrastructure Academy']['Completions'], 0.4, label = 'Successful Completions')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Successful Completions of DCIA Occupational Skills Training FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph2.2_ost_dcia_comp.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
X = ost_2[ost_2.ProgramTitle == 'MPD (Cadet Corps Branch)'].index.tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, ost_2[ost_2.ProgramTitle == 'MPD (Cadet Corps Branch)']['Total'], 0.4, label = 'Total')
plt.bar(X_axis + 0.2, ost_2[ost_2.ProgramTitle == 'MPD (Cadet Corps Branch)']['Completions'], 0.4, label = 'Successful Completions')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Successful Completions of MPD Occupational Skills Training FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph2.2_ost_mpd_comp.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
X = ost_2[ost_2.ProgramTitle == 'Pre-Apprenticeship Program'].index.tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, ost_2[ost_2.ProgramTitle == 'Pre-Apprenticeship Program']['Total'], 0.4, label = 'Total')
plt.bar(X_axis + 0.2, ost_2[ost_2.ProgramTitle == 'Pre-Apprenticeship Program']['Completions'], 0.4, label = 'Successful Completions')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Successful Completions of Pre-App Occupational Skills Training FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph2.2_ost_pre_app_comp.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
X = ost_2[ost_2.ProgramTitle == 'Transitional Employment Program (Way to Work)'].index.tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, ost_2[ost_2.ProgramTitle == 'Transitional Employment Program (Way to Work)']['Total'], 0.4, label = 'Total')
plt.bar(X_axis + 0.2, ost_2[ost_2.ProgramTitle == 'Transitional Employment Program (Way to Work)']['Completions'], 0.4, label = 'Successful Completions')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Successful Completions of TEP Occupational Skills Training FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph2.2_ost_tep_comp.png', dpi=300, bbox_inches='tight')
plt.show()

#### Visualization 2.3: Completion Rates

In [ ]:
plt.style.use('ggplot')

x = ost_2[ost_2.ProgramTitle == 'DC Career Connections (DCCC)'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_2[ost_2.ProgramTitle == 'DC Career Connections (DCCC)']['Completion_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Successful Completions (%)")
plt.title("Successful Completion Rates for DCCC Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.3_ost_dccc_comp_pct.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = ost_2[ost_2.ProgramTitle == 'DC Department of Fire & Emergency Services'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_2[ost_2.ProgramTitle == 'DC Department of Fire & Emergency Services']['Completion_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Successful Completions (%)")
plt.title("Successful Completion Rates for FEMS Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.3_ost_fems_comp_pct.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = ost_2[ost_2.ProgramTitle == 'DC Infrastructure Academy'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_2[ost_2.ProgramTitle == 'DC Infrastructure Academy']['Completion_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Successful Completions (%)")
plt.title("Successful Completion Rates for DCIA Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.3_ost_dcia_comp_pct.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = ost_2[ost_2.ProgramTitle == 'MPD (Cadet Corps Branch)'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_2[ost_2.ProgramTitle == 'MPD (Cadet Corps Branch)']['Completion_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Successful Completions (%)")
plt.title("Successful Completion Rates for MPD Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.3_ost_mpd_comp_pct.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = ost_2[ost_2.ProgramTitle == 'Pre-Apprenticeship Program'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_2[ost_2.ProgramTitle == 'Pre-Apprenticeship Program']['Completion_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Successful Completions (%)")
plt.title("Successful Completion Rates for Pre-App Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.3_ost_pre_app_comp_pct.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = ost_2[ost_2.ProgramTitle == 'Transitional Employment Program (Way to Work)'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_2[ost_2.ProgramTitle == 'Transitional Employment Program (Way to Work)']['Completion_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Successful Completions (%)")
plt.title("Successful Completion Rates for TEP Occupational Skills Training FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph2.3_ost_tep_comp_pct.png', dpi=300, bbox_inches='tight')
plt.show()

## Report 3: GED

#### Filter 1: Program (TEP, DCCC)

In [ ]:
# filtering to relevant programs
ged = ljt[(ljt.ProgramTitle == 'Transitional Employment Program (Way to Work)') | (ljt.ProgramTitle == 'DC Career Connections (DCCC)')]

#### Filter 2: Activity Code (214)

In [ ]:
# filtering by activity code
ged = ged[ged.ServiceCode == '315']

### Report 3.1: Enrollments

#### Filter 3: Start Date

In [ ]:
# filtering by start date
ged_st = ged[ged.StartDate == 1]

In [ ]:
# creating enrollments table
ged_1 = ged_st[['ProgramTitle', 'Completion_num']].groupby('ProgramTitle').count()
ged_1 = ged_1.rename(columns = {"Completion_num": "Enrollments"})
ged_1

#### Visualizations 3.1: Enrollments

In [ ]:
'''
plt.style.use('ggplot')

x = ged_1.index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ost_1['Enrollments'], color='green')
plt.xlabel("Program")
plt.ylabel("Enrollments")
plt.title("Enrollments for GED FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph3.1_ged_enr.png', dpi=300, bbox_inches='tight')
plt.show()
'''

### Report 3.2: Completions

In [ ]:
# filtering by end date
ged_en = ged[ged.EndDate == 1]

In [ ]:
# creating completions table
ged_2 = ged_en[['ProgramTitle', 'Completion_num']].groupby("ProgramTitle").agg(['count', 'sum'])
ged_2.columns = ged_2.columns.get_level_values(1)
ged_2 = ged_2.rename(columns = {"count": "Total", "sum": "Completions"})
ged_2['Completion_Ratio'] = (ged_2.Completions / ged_2.Total)*100
ged_2

#### Visualizations 3.2: Completions

In [ ]:
'''
X = ged_2.index.tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, ged_2['Total'], 0.4, label = 'Total')
plt.bar(X_axis + 0.2, ged_2['Completions'], 0.4, label = 'Successful Completions')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Successful Completions of GED FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph3.2_ged_comp.png', dpi=300, bbox_inches='tight')
plt.show()
'''

#### Visualization 3.3: Completion Rates

In [ ]:
'''
plt.style.use('ggplot')

x = ged_2.index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, ged_2['Completion_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Successful Completions (%)")
plt.title("Successful Completion Rates for GED FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph3.3_ged_comp_pct.png', dpi=300, bbox_inches='tight')
plt.show()
'''

## Report 4: Credentials

#### Filter 1: Program (TEP, DCCC, DCIA)

In [ ]:
# filtering to relevant programs
cred = ljt[(ljt.ProgramTitle == 'Transitional Employment Program (Way to Work)') | (ljt.ProgramTitle == 'DC Career Connections (DCCC)') | (ljt.ProgramTitle == 'DC Infrastructure Academy')]

#### Filter 2: Activity Code (214, 313, 328)

In [ ]:
# filtering by activity code
cred = cred[(cred.ServiceCode == '214') | (cred.ServiceCode == '313') | (cred.ServiceCode == '328')]

### Report 4.1: Enrollments

#### Filter 3: Start Date

In [ ]:
# filtering by start date
cred_st = cred[cred.StartDate == 1]

In [ ]:
# creating enrollments table
cred_1 = cred_st[['Abbreviations', 'Completion_num']].groupby('Abbreviations').count()
cred_1 = cred_1.rename(columns = {"Completion_num": "Enrollments"})
cred_1

#### Visualizations 4.1: Enrollments

In [ ]:
plt.style.use('ggplot')

x = cred_1.index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, cred_1['Enrollments'], color='green')
plt.xlabel("Program")
plt.ylabel("Enrollments")
plt.title("Enrollments for Credentials FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph4.1_cred_enr.png', dpi=300, bbox_inches='tight')
plt.show()

### Report 4.2: Completions

In [ ]:
# filtering by end date
cred_en = cred[cred.EndDate == 1]

In [ ]:
# creating completions table
cred_2 = cred_en[['Abbreviations', 'Completion_num']].groupby("Abbreviations").agg(['count', 'sum'])
cred_2.columns = cred_2.columns.get_level_values(1)
cred_2 = cred_2.rename(columns = {"count": "Total", "sum": "Completions"})
cred_2['Completion_Ratio'] = (cred_2.Completions / cred_2.Total)*100
cred_2

#### Visualizations 4.2: Completions

In [ ]:
X = cred_2.index.tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, cred_2['Total'], 0.4, label = 'Total')
plt.bar(X_axis + 0.2, cred_2['Completions'], 0.4, label = 'Successful Completions')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Successful Completions of Credentials FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph4.2_cred_comp.png', dpi=300, bbox_inches='tight')
plt.show()

#### Visualization 4.3: Completion Rates

In [ ]:
plt.style.use('ggplot')

X_axis = np.arange(len(X))

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, cred_2['Completion_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Successful Completions (%)")
plt.title("Successful Completion Rates for Credentials FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph4.3_cred_comp_pct.png', dpi=300, bbox_inches='tight')
plt.show()

## Report 5: Subsidized Employment

#### Filter 1: Program (TEP, DCCC)

In [ ]:
# filtering to relevant programs
sub = ljt[(ljt.ProgramTitle == 'Transitional Employment Program (Way to Work)') | (ljt.ProgramTitle == 'DC Career Connections (DCCC)')]

#### Filter 2: Activity Code (219)

In [ ]:
# filtering by activity code
sub = sub[sub.ServiceCode == '219']

### Report 5.1: Enrollments

#### Filter 3: Start Date

In [ ]:
# filtering by start date
sub_st = sub[sub.StartDate == 1]

In [ ]:
# creating enrollments table
sub_1 = sub_st[['ProgramTitle', 'StartMonthYear', 'Completion_num']].groupby(['ProgramTitle', 'StartMonthYear']).count()
sub_1 = sub_1.rename(columns = {"Completion_num": "Enrollments"})
sub_1 = sub_1.reset_index(level='ProgramTitle')
sub_1

#### Visualizations 5.1: Enrollments

In [ ]:
plt.style.use('ggplot')

x = sub_1[sub_1.ProgramTitle == 'DC Career Connections (DCCC)'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, sub_1[sub_1.ProgramTitle == 'DC Career Connections (DCCC)']['Enrollments'], color='green')
plt.xlabel("Program")
plt.ylabel("Enrollments")
plt.title("Enrollments for DCCC Subsidized Employment FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph5.1_sub_dccc_enr.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = sub_1[sub_1.ProgramTitle == 'Transitional Employment Program (Way to Work)'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, sub_1[sub_1.ProgramTitle == 'Transitional Employment Program (Way to Work)']['Enrollments'], color='green')
plt.xlabel("Program")
plt.ylabel("Enrollments")
plt.title("Enrollments for TEP Subsidized Employment FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph5.1_sub_tep_enr.png', dpi=300, bbox_inches='tight')
plt.show()

### Report 5.2: Completions

In [ ]:
# filtering by end date
sub_en = sub[sub.EndDate == 1]

In [ ]:
# creating completions table
sub_2 = sub_en[['ProgramTitle', 'StartMonthYear', 'Completion_num']].groupby(["ProgramTitle", "StartMonthYear"]).agg(['count', 'sum'])
sub_2.columns = sub_2.columns.get_level_values(1)
sub_2 = sub_2.rename(columns = {"count": "Total", "sum": "Completions"})
sub_2['Completion_Ratio'] = (sub_2.Completions / sub_2.Total)*100
sub_2 = sub_2.reset_index(level='ProgramTitle')
sub_2

#### Visualizations 5.2: Completions

In [ ]:
X = sub_2[sub_2.ProgramTitle == 'DC Career Connections (DCCC)'].index.tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, sub_2[sub_2.ProgramTitle == 'DC Career Connections (DCCC)']['Total'], 0.4, label = 'Total')
plt.bar(X_axis + 0.2, sub_2[sub_2.ProgramTitle == 'DC Career Connections (DCCC)']['Completions'], 0.4, label = 'Successful Completions')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Successful Completions of DCCC Subsidized Employment FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph5.2_sub_dccc_comp.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
X = sub_2[sub_2.ProgramTitle == 'Transitional Employment Program (Way to Work)'].index.tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, sub_2[sub_2.ProgramTitle == 'Transitional Employment Program (Way to Work)']['Total'], 0.4, label = 'Total')
plt.bar(X_axis + 0.2, sub_2[sub_2.ProgramTitle == 'Transitional Employment Program (Way to Work)']['Completions'], 0.4, label = 'Successful Completions')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Successful Completions of TEP Subsidized Employment FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph5.2_sub_tep_comp.png', dpi=300, bbox_inches='tight')
plt.show()

#### Visualization 5.3: Completion Rates

In [ ]:
plt.style.use('ggplot')

x = sub_2[sub_2.ProgramTitle == 'DC Career Connections (DCCC)'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, sub_2[sub_2.ProgramTitle == 'DC Career Connections (DCCC)']['Completion_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Successful Completions (%)")
plt.title("Successful Completion Rates for DCCC Subsidized Employment FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph5.3_sub_dccc_comp_pct.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.style.use('ggplot')

x = sub_2[sub_2.ProgramTitle == 'Transitional Employment Program (Way to Work)'].index.tolist()

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, sub_2[sub_2.ProgramTitle == 'Transitional Employment Program (Way to Work)']['Completion_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Successful Completions (%)")
plt.title("Successful Completion Rates for TEP Subsidized Employment FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph5.3_sub_tep_comp_pct.png', dpi=300, bbox_inches='tight')
plt.show()

## Report 6: Wages and Retention

In [ ]:
# reading in data
dccc_wage = pd.read_csv('wage_data/DCCCplacementandretentionWages050621.csv')
fems_wage = pd.read_csv('wage_data/FEMSplacementandretentionWages050621.csv')
leap_wage = pd.read_csv('wage_data/LEAPplacementandretentionWages050621.csv')
pre_app_wage = pd.read_csv('wage_data/PreAppplacementandretentionWages050621.csv')
tep_wage = pd.read_csv('wage_data/TEPplacementandretentionWages050621.csv')

In [ ]:
# combining all wage data
wage_data = pd.concat([dccc_wage, fems_wage, leap_wage, pre_app_wage, tep_wage])

In [ ]:
wage_data = wage_data.fillna(0)

In [ ]:
# subsetting
wage_data = wage_data[["State ID", "Program_Name", "2020Q1", "2020Q2", "2020Q3", "2020Q4"]]

In [ ]:
# creating retained column
wage_data['Retained'] = 0

# recoding retained column
wage_data.loc[(wage_data['2020Q1'] != 0) & (wage_data['2020Q2'] != 0), 'Retained'] = 1

In [ ]:
# creating completions table
wage_data = wage_data[['Program_Name', 'Retained']].groupby("Program_Name").agg(['count', 'sum'])
wage_data.columns = wage_data.columns.get_level_values(1)
wage_data = wage_data.rename(columns = {"count": "Total_Exits", "sum": "Retained"})
wage_data['Retained_Ratio'] = (wage_data.Retained / wage_data.Total_Exits)*100
wage_data

#### Visualizations 6.1: Retention

In [ ]:
X = ['FEMS', 'DCCC', 'L.E.A.P.', 'Pre-App', 'TEP']
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, wage_data['Total_Exits'], 0.4, label = 'Total Exited')
plt.bar(X_axis + 0.2, wage_data['Retained'], 0.4, label = 'Total Retained')
  
plt.xticks(X_axis, X)
plt.xlabel("Program")
plt.ylabel("Total Number of Participants")
plt.title("Obtained and Retained Employment FY {} {}".format(year, quarter))
plt.legend()
plt.savefig('graphs/graph6.1_emp_ret.png', dpi=300, bbox_inches='tight')
plt.show()

#### Visualization 6.2: Retention Rates

In [ ]:
plt.style.use('ggplot')

x = ['FEMS', 'DCCC', 'L.E.A.P.', 'Pre-App', 'TEP']

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, wage_data['Retained_Ratio'], color='Blue')
plt.xlabel("Program")
plt.ylabel("Retentioned Employment (%)")
plt.title("Retention Rates for Obtained Employment FY {} {}".format(year, quarter))

plt.xticks(x_pos, x)
plt.savefig('graphs/graph6.1_emp_ret_pct.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# writing out tables
with pd.ExcelWriter('output/tables.xlsx') as writer:  
    jrt_1.to_excel(writer, sheet_name='jrt_enr')
    jrt_2.to_excel(writer, sheet_name='jrt_comp')
    ost_1.to_excel(writer, sheet_name='ost_enr')
    ost_2.to_excel(writer, sheet_name='ost_comp')
    ged_1.to_excel(writer, sheet_name='ged_enr')
    ged_2.to_excel(writer, sheet_name='ged_comp')
    cred_1.to_excel(writer, sheet_name='cred_enr')
    cred_2.to_excel(writer, sheet_name='cred_comp')
    sub_1.to_excel(writer, sheet_name='sub_enr')
    sub_2.to_excel(writer, sheet_name='sub_comp')
    wage_data.to_excel(writer, sheet_name='wage_data')